#### Simple Gen App Using LangChain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
# Data Integration -- from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader=WebBaseLoader("https://bollyflix.faith/")
loader

In [6]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://bollyflix.faith/', 'title': 'BollyFlix | Official Site, Bolly Flix, 300MB Movies, 9xMovies, BollyFlix.Net, BollyFlix.com', 'description': 'Download Latest Bollywood, Hollywood, Dual Audio Hindi Dubbed, 300MB Movies Movies || Google Drive Direct Download Links || BollyFlix, bollyflix, 720p BluRay Movies.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBollyFlix | Official Site, Bolly Flix, 300MB Movies, 9xMovies, BollyFlix.Net, BollyFlix.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nBollyflix\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n🏠 HOME\n🎬 MOVIES\n\nSOUTH HINDI DUBBED\nADULT MOVIES\nBOLLYWOOD\nDUAL AUDIO\nHINDI DUBBED\nHOLLYWOOD\nMULTI AUDIO\nPUNJABI\nBENGALI\n\n\n🔗 GENRE\n\nACTION\nADVENTURE\nANIMATION\nCOMEDY\nCRIME\nDRAMA\nFANTASY\nHISTORICAL\nHORROR\nMYSTERY\nPOLITICAL\nROMANCE\nSCI-FI\nTHRILLER\n\n\n📅 YEAR\n\n2

In [7]:
### load data --> docs --> divide our text into chunks --> vectors --> vector embeddings --> vector storeDB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://bollyflix.faith/', 'title': 'BollyFlix | Official Site, Bolly Flix, 300MB Movies, 9xMovies, BollyFlix.Net, BollyFlix.com', 'description': 'Download Latest Bollywood, Hollywood, Dual Audio Hindi Dubbed, 300MB Movies Movies || Google Drive Direct Download Links || BollyFlix, bollyflix, 720p BluRay Movies.', 'language': 'en-US'}, page_content='BollyFlix | Official Site, Bolly Flix, 300MB Movies, 9xMovies, BollyFlix.Net, BollyFlix.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nBollyflix\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n🏠 HOME\n🎬 MOVIES\n\nSOUTH HINDI DUBBED\nADULT MOVIES\nBOLLYWOOD\nDUAL AUDIO\nHINDI DUBBED\nHOLLYWOOD\nMULTI AUDIO\nPUNJABI\nBENGALI\n\n\n🔗 GENRE\n\nACTION\nADVENTURE\nANIMATION\nCOMEDY\nCRIME\nDRAMA\nFANTASY\nHISTORICAL\nHORROR\nMYSTERY\nPOLITICAL\nROMANCE\nSCI-FI\nTHRILLER\n\n\n📅 YEAR\n\n2025\n2024\n2023\n2022\n2021\n2016-2020\n\n2016\n2017\n2018

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="deepseek-r1:1.5b")


C:\Users\HP\AppData\Local\Temp\ipykernel_6508\1357735585.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="deepseek-r1:1.5b")


In [10]:
# # ...existing code...
# from langchain_community.embeddings import OllamaEmbeddings

# # Use a free, local embedding model (e.g., "phi" or "mistral")
# embeddings = OllamaEmbeddings(model="phi")  # or "mistral", "llama2:7b", etc.

# from langchain_community.vectorstores import FAISS
# vectorstoredb = FAISS.from_documents(documents, embeddings)
# # ...existing code...

In [11]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [12]:
vectorstoredb

In [13]:
# write a query from the website
query="download Coolie (2025) Hindi Movie available in 1080p, 720p & 480p Qualities For Your Mobile/tablet/Computer. This movie is based on Action."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Download Blackjack (1998) REMASTERED UNCUT Dual Audio [Hindi-English] Movie 480p | 720p | 1080p BluRay ESub \n\n\n\n \n\n\nDownload Abandoned Man (2025) Dual Audio [Hindi-Turkish] Movie 480p | 720p | 1080p WEB-DL MSubs \n\n\n\n \n\n\nDownload Two Graves (Season 1) Multi Audio [Hindi-English-Spanish] WEB Series 480p | 720p | 1080p WEB-DL MSubs \n\n\n\n \n\n\nDownload Dr. Stone (Season 1-4) Multi Audio [Hindi-English-Japanese] Anime Series 480p | 720p | 1080p WEB-DL MSubs [S04E20 Added] \n\n\n\n \n\n\nDownload Lord of the Mysteries (Season 1) Dual Audio {Hindi-Chinese} Anime Series 480p | 720p | 1080p Web-DL ESub [S01E08 Added] \n\n\n\n \n\n\nDownload Zebra (2024) Dual Audio [Hindi-Telugu] Movie 480p | 720p | 1080p WEB-DL ESub \n\n\n\n \n\n\nDownload Vunnadhi Okate Zindagi (2017) Dual Audio [Hindi-Telugu] Movie 480p | 720p | 1080p WEB-DL ESub \n\n\n\n \n\n\nDownload Vanangaan (2025) Dual Audio [Hindi-Tamil] Movie 480p | 720p | 1080p WEB-DL ESub \n \n\nPosts pagination\n1\n2\n3\n4\n5\n6\

In [14]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="deepseek-r1:1.5b")

C:\Users\HP\AppData\Local\Temp\ipykernel_6508\1077620050.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="deepseek-r1:1.5b")


In [15]:
## retrival chain, documents chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    
    """
)

document_chain=create_stuff_documents_chain(llm,prompt)


In [16]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n\n    '), additional_kwargs={})])
| ChatOllama(model='deepseek-r1:1.5b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"download Coolie (2025) Hindi Movie available in 1080p, 720p & 480p Qualities For Your Mobile/tablet/Computer. This movie is based on Action.",
    "context":[Document(page_content="Download Coolie (2025) Hindi Movie 480p | 720p | 1080p HDTC 1 week ago✅Download Coolie (2025) Hindi Movie available in 1080p, 720p & 480p Qualities For Your Mobile/tablet/Computer. This movie is based on Action.")]
    
})

'<think>\nOkay, I need to answer a question based on the provided context about a movie called "Coolie (2025)". The user\'s query is in the form of a fill-in-the-blank like format: "Download Coolie (2025) Hindi Movie 480p | 720p | 1080p HDTC 1 week ago✅...". So, I should extract the relevant details from the context.\n\nLooking at the context, it mentions that the movie is available in 480p, 720p, and 1080p for various devices with HDTC. The user probably wants to know which formats are supported, maybe to download or stream the movie on their device.\n\nI\'ll structure the answer by listing each format followed by its size and whether it\'s HDTC. That should help anyone who needs to access the movie on different screens.\n</think>\n\nThe context indicates that the movie "Coolie (2025)" is available in 480p, 720p, and 1080p resolutions. Since the movie is described as based on Action, it\'s likely playable on mobile devices. The HDTC support means it should be accessible on various HD 

However we want the document to first come from the retriver we just set up. That way, we can use the retriver to dinamically select the most relavent document and pass those in for a given question.

In [18]:
### Input --> Retriver --> vectorstoreDB

vectorstoredb

In [19]:
retriver=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrival_chain=create_retrieval_chain(retriver,document_chain)

In [20]:
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017DFC94B3D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n\n    '), additional_kwargs={}

In [21]:
## get the response from the LLM
response=retrival_chain.invoke({"input":"This movie is based on Action."})
response['answer']

'<think>\nOkay, so I\'m trying to figure out what the user is asking for. They provided a context with several movie files and some information about BollyFlix. It seems like they\'re looking for a way to view these movies online.\n\nFirst, I\'ll go through each file in the context. The first one is "Blackjack (1998) REMASTERED UNCUT Dual Audio [Hindi-English] Movie 480p | 720p | 1080p BluRay ESub." That\'s a blackjack movie, so I know it\'s probably an action or drama film.\n\nNext is "Abandoned Man (2025) Dual Audio [Hindi-Turkish] Movie 480p | 720p | 1080p WEB-DL MSubs." That\'s a Turkish movie from 2025. Maybe it\'s set in an abandoned area.\n\nThen there are several "Two Graves" entries, all Multi Audio [Hindi-English-Spanish] and Web Series. So these could be spanning multiple languages, maybe from India or other South Asian countries. The URLs like 480p | 720p | 1080p WEB-DL MSubs suggest they\'re available on different web series platforms.\n\n"Dr. Stone (Season 1-4) Multi Audi

In [22]:
print(response)

{'input': 'This movie is based on Action.', 'context': [Document(id='745731cd-cdb9-4114-8ae3-874531b4ceba', metadata={'source': 'https://bollyflix.faith/', 'title': 'BollyFlix | Official Site, Bolly Flix, 300MB Movies, 9xMovies, BollyFlix.Net, BollyFlix.com', 'description': 'Download Latest Bollywood, Hollywood, Dual Audio Hindi Dubbed, 300MB Movies Movies || Google Drive Direct Download Links || BollyFlix, bollyflix, 720p BluRay Movies.', 'language': 'en-US'}, page_content='Download Blackjack (1998) REMASTERED UNCUT Dual Audio [Hindi-English] Movie 480p | 720p | 1080p BluRay ESub \n\n\n\n \n\n\nDownload Abandoned Man (2025) Dual Audio [Hindi-Turkish] Movie 480p | 720p | 1080p WEB-DL MSubs \n\n\n\n \n\n\nDownload Two Graves (Season 1) Multi Audio [Hindi-English-Spanish] WEB Series 480p | 720p | 1080p WEB-DL MSubs \n\n\n\n \n\n\nDownload Dr. Stone (Season 1-4) Multi Audio [Hindi-English-Japanese] Anime Series 480p | 720p | 1080p WEB-DL MSubs [S04E20 Added] \n\n\n\n \n\n\nDownload Lord o